In [1]:
import sklearn

**Code is licensed under MIT license! Visit https://github.com/newcodevelop/DL-Adventures/blob/master/multilingual_bert_infomax(pre_final).ipynb :)**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [2]:
!nvidia-smi

Tue Jul  7 17:16:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%tensorflow_version 1.x
import tensorflow as tf

TensorFlow 1.x selected.


In [4]:
!pip install bert-tensorflow
import bert
from bert import run_classifier

from bert import optimization
from bert import tokenization

     |████████████████████████████████| 71kB 2.0MB/s 



In [10]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip

--2020-07-07 17:18:59--  https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.125.128, 74.125.23.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 404261442 (386M) [application/zip]
Saving to: ‘cased_L-12_H-768_A-12.zip’

cased_L-12_H-768_A- 100%[===================>] 385.53M  53.4MB/s    in 7.3s    

2020-07-07 17:19:06 (52.5 MB/s) - ‘cased_L-12_H-768_A-12.zip’ saved [404261442/404261442]



In [11]:
!ls

cased_L-12_H-768_A-12.zip  sample_data	snli_1.0.zip


In [6]:
!wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip

--2020-07-07 17:17:55--  https://nlp.stanford.edu/projects/snli/snli_1.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94550081 (90M) [application/zip]
Saving to: ‘snli_1.0.zip’

snli_1.0.zip        100%[===================>]  90.17M  20.9MB/s    in 7.1s    

2020-07-07 17:18:03 (12.8 MB/s) - ‘snli_1.0.zip’ saved [94550081/94550081]



In [12]:
!unzip cased_L-12_H-768_A-12.zip 
!unzip snli_1.0.zip

Archive:  cased_L-12_H-768_A-12.zip
   creating: cased_L-12_H-768_A-12/
  inflating: cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: cased_L-12_H-768_A-12/vocab.txt  
  inflating: cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: cased_L-12_H-768_A-12/bert_config.json  
Archive:  snli_1.0.zip
   creating: snli_1.0/
  inflating: snli_1.0/.DS_Store      
   creating: __MACOSX/
   creating: __MACOSX/snli_1.0/
  inflating: __MACOSX/snli_1.0/._.DS_Store  
 extracting: snli_1.0/Icon           
  inflating: __MACOSX/snli_1.0/._Icon  
  inflating: snli_1.0/README.txt     
  inflating: __MACOSX/snli_1.0/._README.txt  
  inflating: snli_1.0/snli_1.0_dev.jsonl  
  inflating: snli_1.0/snli_1.0_dev.txt  
  inflating: snli_1.0/snli_1.0_test.jsonl  
  inflating: snli_1.0/snli_1.0_test.txt  
  inflating: snli_1.0/snli_1.0_train.jsonl  
  inflating: snli_1.0/snli_1.0_train.txt  
  inflating: __MACOSX/._snli_1.0 

In [14]:
import pandas as pd

In [18]:
df = pd.read_csv('snli_1.0/snli_1.0_train.txt',sep = '\t')

In [19]:
df.head(10)

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5
0,neutral,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,3416050480.jpg#4,3416050480.jpg#4r1n,neutral,NaN,NaN,NaN,NaN
1,contradiction,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",3416050480.jpg#4,3416050480.jpg#4r1c,contradiction,NaN,NaN,NaN,NaN
2,entailment,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,"( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",3416050480.jpg#4,3416050480.jpg#4r1e,entailment,NaN,NaN,NaN,NaN
3,neutral,( Children ( ( ( smiling and ) waving ) ( at c...,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,Children smiling and waving at camera,They are smiling at their parents,2267923837.jpg#2,2267923837.jpg#2r1n,neutral,NaN,NaN,NaN,NaN
4,entailment,( Children ( ( ( smiling and ) waving ) ( at c...,( There ( ( are children ) present ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,Children smiling and waving at camera,There are children present,2267923837.jpg#2,2267923837.jpg#2r1e,entailment,NaN,NaN,NaN,NaN
5,contradiction,( Children ( ( ( smiling and ) waving ) ( at c...,( ( The kids ) ( are frowning ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (DT The) (NNS kids)) (VP (VBP are...,Children smiling and waving at camera,The kids are frowning,2267923837.jpg#2,2267923837.jpg#2r1c,contradiction,NaN,NaN,NaN,NaN
6,contradiction,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,( ( The boy ) ( ( ( skates down ) ( the sidewa...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ skate...,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,3691670743.jpg#0,3691670743.jpg#0r1c,contradiction,NaN,NaN,NaN,NaN
7,entailment,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,( ( The boy ) ( ( does ( a ( skateboarding tri...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ does)...,A boy is jumping on skateboard in the middle o...,The boy does a skateboarding trick.,3691670743.jpg#0,3691670743.jpg#0r1e,entailment,NaN,NaN,NaN,NaN
8,neutral,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,( ( The boy ) ( ( is ( wearing ( safety equipm...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ is) (...,A boy is jumping on skateboard in the middle o...,The boy is wearing safety equipment.,3691670743.jpg#0,3691670743.jpg#0r1n,neutral,NaN,NaN,NaN,NaN
9,neutral,( ( An ( older man ) ) ( ( ( sits ( with ( ( h...,( ( An ( older man ) ) ( ( ( drinks ( his juic...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...,An older man sits with his orange juice at a s...,An older man drinks his juice as he waits for ...,4804607632.jpg#0,4804607632.jpg#0r1n,neutral,NaN,NaN,NaN,NaN


In [33]:
from sklearn.model_selection import train_test_split
df = df.head(50000)
train, test = train_test_split(df, test_size=0.1,random_state = 42,shuffle = True)

In [34]:
def get_data_eng_eng(a):
  b = list(a['gold_label'])
  lab = []
  
  for i in b:
    if i=='contradiction':
        lab.append(0)
    elif i=='neutral':
        lab.append(1)
    elif i== 'entailment':
        lab.append(2)
    else:
        lab.append(3)
  sentence_1 = list(a['sentence1'])
  sentence_2 = list(a['sentence2'])
  raw_data_train = {'sentence1_eng': sentence_1, 
              'sentence2_eng': sentence_2,
          'label': lab}
  df = pd.DataFrame(raw_data_train, columns = ['sentence1_eng','sentence2_eng','label'])
  return df

In [35]:
train_eng_eng = get_data_eng_eng(train)


In [36]:
test_eng_eng = get_data_eng_eng(test)


In [37]:
test_labels = list(test_eng_eng['label'])

In [38]:
train_eng_eng[0:10]

,sentence1_eng,sentence2_eng,label
0,A man in a baseball cap cleans out a blue colo...,A man watches the baseball game.,0
1,Man wearing jeans and black shirt rides down t...,This man is wearing no safety gear,1
2,The speaker is introducing the new products on...,a speaker does not like the products he presents,1
3,People hangout on a beach.,A group of friends are watching TV on the couch,0
4,A man and a woman dressed in jeans and white s...,the two men are staring out across the flat de...,0
5,A young boy wearing a blue sweater holding up ...,there is a boy in blue sweater holding necklac...,2
6,"a black and white dog jumping over a red, whit...",The dog is at a jumping competition.,1
7,A child jumping off bleachers with a blue shirt.,A child jumping.,2
8,A person with a green hat laying down on the g...,A person is sleep.,1
9,A young man with a mohawk and a black Dio shir...,A young man on his cell phone.,2


In [39]:
test_eng_eng[0:10]

,sentence1_eng,sentence2_eng,label
0,A rocky shore with two young girls walking and...,Two girls looking for a place to build a sand ...,1
1,Two people stand by the water's edge.,A group of golfers are trying to retrieve a ba...,1
2,People on bicycles waiting at an intersection.,People are riding their bicycles.,2
3,Two men dressed in blue uniforms wearing headp...,Two police officers wait on for a response call.,1
4,A woman cooks on the stove in her kitchen as h...,A woman's dog is whining for the food the woma...,2
5,A baseball batter in white pants and a black s...,The batter is wearing white pants.,2
6,A person in a red vest walks past a shuttered ...,A person in blue walks into a door.,0
7,A boy with face paint is running on a track.,"A girl sleeps motionlessly, alone.",0
8,A young man in a tan hat and a green shirt pee...,A man is looking at his house.,1
9,A group of people sitting against a wall eat s...,The people are eating.,2


In [40]:
label_list = [0,1,2,3]

In [41]:
train_InputExamples_eng = train_eng_eng.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x['sentence1_eng'], 
                                                                   text_b = x['sentence2_eng'], 
                                                                   label = x['label']), axis = 1)


In [42]:
test_InputExamples_eng = test_eng_eng.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x['sentence1_eng'], 
                                                                   text_b = x['sentence2_eng'], 
                                                                   label = x['label']), axis = 1)
                                                                

In [43]:
vocab_file = "cased_L-12_H-768_A-12/vocab.txt"
def create_tokenizer_from_hub_module():
 
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=True)

tokenizer = create_tokenizer_from_hub_module()

In [44]:
tokenizer.tokenize("how are you")

['how', 'are', 'you']

In [45]:
MAX_SEQ_LENGTH = 128
#MAX_SEQ_LENGTH = 92
# Convert our train and test features to InputFeatures that BERT understands.
train_features_eng = bert.run_classifier.convert_examples_to_features(train_InputExamples_eng, label_list, MAX_SEQ_LENGTH, tokenizer)
#train_features_hindi = bert.run_classifier.convert_examples_to_features(train_InputExamples_hindi, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 45000
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] a man in a baseball cap clean ##s out a blue colored pit outside where colorful ch ##rist ##mas lights hang above him . [SEP] a man watches the baseball game . [SEP]
INFO:tensorflow:input_ids: 101 170 1299 1107 170 3866 6707 4044 1116 1149 170 2221 6805 7172 1796 1187 15302 22572 12937 7941 4204 7311 1807 1140 119 102 170 1299 12642 1103 3866 1342 119 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0

In [47]:
MAX_SEQ_LENGTH = 128
#MAX_SEQ_LENGTH = 92
# Convert our train and test features to InputFeatures that BERT understands.
test_features_eng = bert.run_classifier.convert_examples_to_features(test_InputExamples_eng, label_list, MAX_SEQ_LENGTH, tokenizer)
#test_features_hindi = bert.run_classifier.convert_examples_to_features(test_InputExamples_hindi, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 5000
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] a rocky shore with two young girls walking and a lady pushing a man in a wheelchair . [SEP] two girls looking for a place to build a sand castle . [SEP]
INFO:tensorflow:input_ids: 101 170 10987 5781 1114 1160 1685 2636 3179 1105 170 5141 5859 170 1299 1107 170 14980 119 102 1160 2636 1702 1111 170 1282 1106 3076 170 5387 3804 119 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [82]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
  """Creates a classification model."""
  model = bert.run_classifier.modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)

  # In the demo, we are doing a simple classification task on the entire
  # segment.
  #
  # If you want to use the token-level output, use model.get_sequence_output()
  # instead.
  output_layer = model.get_pooled_output()
  print(output_layer.shape)
  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)
    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, per_example_loss, logits, probabilities,predicted_labels,output_layer)


In [92]:
def create_model_modified(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
  """Creates a classification model."""
  model = bert.run_classifier.modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)

  # In the demo, we are doing a simple classification task on the entire
  # segment.
  #
  # If you want to use the token-level output, use model.get_sequence_output()
  # instead.
  #output_layer = model.get_pooled_output()
  output_layer = model.get_sequence_output()
  hidden_size = output_layer.shape[2].value
  max_len = output_layer.shape[1].value
  print(hidden_size)
  print(max_len)
  assert max_len==128
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())
  mod_weights = tf.get_variable(
      "mod_weights", [1, max_len],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    output_layer = tf.matmul(mod_weights,output_layer)
    print(f'output_layer_shape {output_layer.shape}')
    output_layer = tf.squeeze(output_layer)
    print(f'output_layer_shape modified{output_layer.shape}')
    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)
    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, per_example_loss, logits, probabilities,predicted_labels,output_layer)


In [93]:
import numpy as np

In [94]:
def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]
    is_real_example = None
    if "is_real_example" in features:
      is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
    else:
      is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities,predicted_labels,hidden_context) = create_model(
        bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
        num_labels, use_one_hot_embeddings)

    tvars = tf.trainable_variables()
    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names
      ) = bert.run_classifier.modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:

      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.estimator.EstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op)
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits, is_real_example):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        accuracy = tf.metrics.accuracy(
            labels=label_ids, predictions=predictions, weights=is_real_example)
        loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
       
        return {
            "eval_accuracy": accuracy,
            "eval_loss": loss
        }

      eval_metrics = metric_fn(per_example_loss, label_ids, logits, is_real_example)
      
      output_spec = tf.estimator.EstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metric_ops=eval_metrics)
    else:
      output_spec = tf.estimator.EstimatorSpec(
          mode=mode,
          predictions={"probabilities": probabilities,"labels": predicted_labels, "hidden_context": hidden_context})
    return output_spec

  return model_fn



In [95]:
def model_fn_builder_modified(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]
    is_real_example = None
    if "is_real_example" in features:
      is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
    else:
      is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities,predicted_labels,hidden_context) = create_model_modified(
        bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
        num_labels, use_one_hot_embeddings)

    tvars = tf.trainable_variables()
    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names
      ) = bert.run_classifier.modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:

      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.estimator.EstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op)
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits, is_real_example):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        accuracy = tf.metrics.accuracy(
            labels=label_ids, predictions=predictions, weights=is_real_example)
        loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
       
        return {
            "eval_accuracy": accuracy,
            "eval_loss": loss
        }

      eval_metrics = metric_fn(per_example_loss, label_ids, logits, is_real_example)
      
      output_spec = tf.estimator.EstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metric_ops=eval_metrics)
    else:
      output_spec = tf.estimator.EstimatorSpec(
          mode=mode,
          predictions={"probabilities": probabilities,"labels": predicted_labels, "hidden_context": hidden_context})
    return output_spec

  return model_fn



In [96]:
def input_fn_builder(features, hidden_context,seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)
    hidden_shape = hidden_context.shape[-1]
    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),

        "hidden_context":
            tf.constant(hidden_context, shape = [num_examples,hidden_shape], dtype = tf.float32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn



In [97]:
def train(ctx,output_dir,input_fn):


  CONFIG_FILE = "cased_L-12_H-768_A-12/bert_config.json"
  INIT_CHECKPOINT = "cased_L-12_H-768_A-12/bert_model.ckpt"

  BATCH_SIZE = 28
  LEARNING_RATE = 2e-5
  NUM_TRAIN_EPOCHS = 1
  # Warmup is a period of time where hte learning rate 
  # is small and gradually increases--usually helps training.
  WARMUP_PROPORTION = 0.1
  # Model configs
  SAVE_CHECKPOINTS_STEPS = 6000
  SAVE_SUMMARY_STEPS = 100
  OUTPUT_DIR = output_dir
  # Compute # train and warmup steps from batch size
  num_train_steps = int(len(input_fn) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
  num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
  print(num_train_steps)
  run_config = tf.estimator.RunConfig(
      model_dir=OUTPUT_DIR,
      save_summary_steps=SAVE_SUMMARY_STEPS,
      save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

  # Specify outpit directory and number of checkpoint steps to save
 
  

  if ctx == 0:
    model_fn = model_fn_builder(
      bert_config=bert.run_classifier.modeling.BertConfig.from_json_file(CONFIG_FILE),
      num_labels=4, #number of unique labels
      init_checkpoint=INIT_CHECKPOINT,
      learning_rate=LEARNING_RATE,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      use_tpu=False,
      use_one_hot_embeddings=False
    )
  else:
    model_fn = model_fn_builder_modified(
      bert_config=bert.run_classifier.modeling.BertConfig.from_json_file(CONFIG_FILE),
      num_labels=4, #number of unique labels
      init_checkpoint=INIT_CHECKPOINT,
      learning_rate=LEARNING_RATE,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      use_tpu=False,
      use_one_hot_embeddings=False
    )
    


  estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    config=run_config,
    params={"batch_size": BATCH_SIZE})



  train_input_fn = bert.run_classifier.input_fn_builder(
      features=input_fn,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=False)
  


  print(f'Beginning Training!')
  %timeit

  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  return estimator


In [98]:
'''
this function evaluates and gets hidden context on english premise and hindi hypo and returns labels for f1 score
'''

def evaluate(estimator,input_fn_for_test):
  MAX_SEQ_LENGTH = 128
 
  
  test_input_fn = run_classifier.input_fn_builder(
    features=input_fn_for_test,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)
  actual_labels = []
  
  for i in test_eng_eng['label']:
    actual_labels.append(i)
  

  res = estimator.predict(test_input_fn)
  predicted_labels = []

  for i in res:
    predicted_labels.append(i['labels'])

  estimator.evaluate(input_fn=test_input_fn, steps=None)
  
    
  return actual_labels,predicted_labels

In [99]:
'''
this function trains on english premise and eng hypothesis
'''
estimator = train(0,'out_dir_train_eng',train_features_eng)


1607
INFO:tensorflow:Using config: {'_model_dir': 'out_dir_train_eng', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 6000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f07c7156320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Beginning Training!
INFO:tensorflow:Skipping training since max_steps has already saved

In [ ]:
!cp -r 'out_dir_train_eng/' '/content/gdrive/My Drive/monolingual'

In [100]:


#hidden_context_eng = evaluate_and_get_hidden_context(estimator,input_fn_for_test = test_features_eng,input_fn_for_hidden = train_features_eng,is_progressive = False)
act_lab,pred_lab = evaluate(estimator,test_features_eng)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
(?, 768)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (28996, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/

In [101]:
'''
this function trains on english premise and eng hypothesis
'''
estimator = train(1,'out_dir_train_eng_modified',train_features_eng)


1607
INFO:tensorflow:Using config: {'_model_dir': 'out_dir_train_eng_modified', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 6000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f083999f2b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Beginning Training!
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:*** Feat

In [102]:


#hidden_context_eng = evaluate_and_get_hidden_context(estimator,input_fn_for_test = test_features_eng,input_fn_for_hidden = train_features_eng,is_progressive = False)
act_lab,pred_lab = evaluate(estimator,test_features_eng)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
768
128
output_layer_shape (?, 1, 768)
output_layer_shape modified<unknown>
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (28996, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
